In [1]:
from stock_prediction.data_prep import Data_Prep
from stock_prediction.features_exo import exo_selection
from stock_prediction.garch import garch
from stock_prediction.arima import arima_multi_day

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from arch import arch_model
from arch.__future__ import reindexing

/Users/arnaudraybaud/code/valouchou/Jaouad-AHRAOUI/stock_prediction/stock_prediction/garch.py:6: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime
<ipython-input-1-23e52b822529>:11: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [57]:
stock = Data_Prep('vinci', 20)

In [58]:
data_global = stock.data_prep()

In [59]:
data_global = data_exo = stock.select_features(data_global,
                                                   Return=True,
                                                   Log_Return=False,
                                                   High_Low=False,
                                                   High_Close=False,
                                                   Low_Close=False,
                                                   Volume_Change=False,
                                                   Period_Volum=False,
                                                   Annual_Vol=False,
                                                   Period_Vol=False,
                                                   Return_Index=False,
                                                   Volum_Index=False,
                                                   Relative_Return=False)

In [60]:
y_returns = data_global.drop(columns='Date')

In [61]:
y_returns = y_returns * 100

In [62]:
y_returns

,Return_DG.PA
0,1.554749
1,-0.218706
2,0.683328
3,-0.371348
4,-1.323909
...,...
1022,-0.513923
1023,1.119243
1024,-0.095781
1025,-0.447434


In [81]:
rolling_predictions = []

In [82]:
split = 50

In [83]:
for i in range(split, len(y_returns)) :
        train = y_returns[ i - split : i]
        # we instatiate the model on the split days
        # we will see if we change the parameters
        # for the moment we keep the default parameters
        # mean = ‘Constant’, ‘Zero’, ‘LS’, ‘AR’, ‘ARX’, ‘HAR’ and ‘HARX’
        # vol = ‘GARCH’ (default), ‘ARCH’, ‘EGARCH’, ‘FIARCH’ and ‘HARCH’
        # dist = Normal , Students, skewed Students, generalized error
        model_garch = arch_model(train, vol='Garch', p=1, q=1, dist='Normal', mean='Constant')
        # we fit on the last year / rows available
        model_fit = model_garch.fit(disp='off')
        pred = model_fit.forecast(horizon=1)

        # we store the result (variance of residuals)
        rolling_predictions.append(np.sqrt(pred.variance.values[0][0]))

In [84]:
rolling_predictions = np.array(rolling_predictions) * np.sqrt(252)

In [85]:
len(rolling_predictions)

977

In [86]:
garch_pred = pd.DataFrame({'garch_pred' : rolling_predictions,
                          'Date' : data_global['Date'][50:]})

In [87]:
garch_pred

,garch_pred,Date
50,20.385430,2017-08-09
51,20.005541,2017-08-10
52,19.451296,2017-08-11
53,19.434636,2017-08-14
54,19.634553,2017-08-15
...,...,...
1022,15.917519,2021-05-31
1023,16.442224,2021-06-01
1024,16.933614,2021-06-02
1025,13.964416,2021-06-03


In [40]:
model_fit = model_garch.fit(update_freq= 5, disp='off')

In [46]:
print(model_fit.summary())

                     Constant Mean - GARCH Model Results                      
Dep. Variable:           Return_DG.PA   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -1824.91
Distribution:                  Normal   AIC:                           3657.81
Method:            Maximum Likelihood   BIC:                           3677.55
                                        No. Observations:                 1027
Date:                Mon, Jun 14 2021   Df Residuals:                     1026
Time:                        14:14:32   Df Model:                            1
                                Mean Model                                
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0975  3.744e-02      2.605  9.197e-03 [2.414e-0

In [49]:
model_fit = model_garch.fit(update_freq= 10, disp='off')

In [50]:
print(model_fit.summary())

                     Constant Mean - GARCH Model Results                      
Dep. Variable:           Return_DG.PA   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -1824.91
Distribution:                  Normal   AIC:                           3657.81
Method:            Maximum Likelihood   BIC:                           3677.55
                                        No. Observations:                 1027
Date:                Mon, Jun 14 2021   Df Residuals:                     1026
Time:                        14:58:38   Df Model:                            1
                                Mean Model                                
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0975  3.744e-02      2.605  9.197e-03 [2.414e-0

In [53]:
pred = model_fit.forecast(horizon=1)

In [54]:
pred

In [56]:
pred.variance.values[0][0]

0.9118831683820805

In [3]:
garch('vinci', 100)

,garch_pred,Date,vol_variation
927,27.175907,2021-01-14,0.015745
928,27.179607,2021-01-15,0.000136
929,27.076088,2021-01-18,-0.003809
930,26.940328,2021-01-19,-0.005014
931,26.766648,2021-01-20,-0.006447
...,...,...,...
1022,14.039886,2021-05-31,-0.029189
1023,13.084131,2021-06-01,-0.068074
1024,13.287345,2021-06-02,0.015531
1025,13.311260,2021-06-03,0.001800


In [3]:
dict_test = { 'a' : 2 , 'b' : 4}
dict_test['a']

2

In [2]:
arima_multi_day('vinci', 100, 0.05) 

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/bas

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/arnaudraybaud/.pyenv/versions/3.8.6/envs/stock_prediction/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWa

,yesterday,prediction,conf_low,conf_high,true,conf_std,Date,perf_pred,perf_true,perf_low,perf_high
41,96.730367,99.165067,96.969493,101.360640,99.608567,1.120211,2021-01-14,0.025170,0.029755,0.002472,0.047868
42,99.608567,98.424367,96.309395,100.539338,97.536272,1.079087,2021-01-15,-0.011889,-0.020804,-0.033121,0.009344
43,97.536272,98.501159,96.370970,100.631348,97.467197,1.086851,2021-01-18,0.009893,-0.000708,-0.011947,0.031733
44,97.467197,97.038341,94.902645,99.174038,97.674416,1.089661,2021-01-19,-0.004400,0.002126,-0.026312,0.017512
45,97.674416,96.057207,93.913415,98.200999,96.223813,1.093792,2021-01-20,-0.016557,-0.014851,-0.038505,0.005391
...,...,...,...,...,...,...,...,...,...,...,...
136,109.965611,109.166797,107.416075,110.917520,109.400472,0.893242,2021-05-31,-0.007264,-0.005139,-0.023185,0.008656
137,109.400472,109.861505,108.275572,111.447439,110.624930,0.809165,2021-06-01,0.004214,0.011192,-0.010282,0.018711
138,110.624930,111.006547,109.415507,112.597587,110.518972,0.811770,2021-06-02,0.003450,-0.000958,-0.010933,0.017832
139,110.518972,110.559108,108.977372,112.140843,110.024473,0.807023,2021-06-03,0.000363,-0.004474,-0.013949,0.014675
